In [1]:
from pyspark.sql import Row
from pyspark.sql.functions import col 
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import RandomForestClassifier

from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
import time

In [2]:
base_path = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_prediction/"

train_positive_pairs = sc.textFile(base_path + "new_positive_pairs_features/day*/part-*").map(lambda x: (eval(x)[0], [abs(i) for y in eval(x)[1] for i in y ]))
train_unlabled_pairs = sc.textFile(base_path + "new_unlabled_pairs_features/part-*").map(lambda x: (eval(x)[0], [abs(i) for y in eval(x)[1] for i in y ]))

test_positive_pairs = sc.textFile(base_path + "test_positive_pairs/part-*")

# test_pairs_features = sc.textFile(base_path + "new_test_pairs_features/part-*").map(lambda x: (eval(x)[0], [abs(i) for y in eval(x)[1] for i in y ]))
# test_pairs_features_50w = sc.textFile(base_path + "test_pairs_features_50w/part-*").map(lambda x: eval(x))

In [4]:
test_positive_pairs_list = test_positive_pairs.collect()   # 第四天数据中的正例对
test_data_4thday = test_pairs_features_50w.map(lambda x: Row(label=1.0 if x[0] in test_positive_pairs_list else 0.0,\
                                                         features=Vectors.dense(x[1]))).toDF()

In [5]:
y_label_noPU = [i.label for i in test_data_4thday.select('label').collect()]

In [3]:
test_pos_features = sc.textFile(base_path + "test_pos_features/part-*").map(lambda x: eval(x))
test_neg_features = sc.textFile(base_path + "test_neg_features_8w/part-*").map(lambda x: eval(x))
test_data_4thday = test_pos_features.map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
        .union(test_neg_features.map(lambda x: Row(label=0.0, features=Vectors.dense(x[1])))).toDF()

In [4]:
y_label_noPU = [i.label for i in test_data_4thday.select('label').collect()]

In [5]:
test_data_4thday_length = len(y_label_noPU)
print(test_data_4thday_length)

157653


In [6]:
for alfa in [0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40]: #, 0.20]:#, 0.30]:#, 0.35, 0.40]: # 0.05, , 0.25
    with open('qq_noPU_alfa_v3331.log', 'a+') as log_file:
        log_file.write('===========predict whitout PU=========== \n')
        log_file.write(time.asctime() + '\n')
        log_file.write( "++++++++++alfa: " + str(alfa) + "+++++++++++++\n")
        print(time.asctime())
        print("alfa: " + str(alfa))
        # train_unlabeled -> train_neg
        #train_negative_pairs = sc.parallelize(train_unlabled_pairs.takeSample(False, int(alfa * 8268837)))
        #train_negative_pairs.coalesce(20).saveAsTextFile(base_path + "wbb_train_negative_pairs-" + str(alfa))
        train_negative_pairs = sc.textFile(base_path + "wbb_train_negative_pairs-" + str(alfa)).map(lambda x: eval(x))

        log_file.write("length of train_negative_pairs: " + str(train_negative_pairs.count()) + '\n')
        print(time.asctime())
        log_file.write(time.asctime() + '\n')

        #============ without PU =======================
        # predict without PU
        train_data_noPU = train_positive_pairs.map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
        .union(train_negative_pairs.map(lambda x: Row(label=0.0, features=Vectors.dense(x[1])))).toDF()
        
        """train_data_noPU = sc.union([train_positive_pairs]*int(alfa/0.015)).map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
        .union((train_negative_pairs.map(lambda x: Row(label=0.0, features=Vectors.dense(x[1]))))).toDF()"""

        """train_data_noPU = train_positive_pairs.map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
        .union(sc.parallelize(train_negative_pairs.takeSample(False, int(100000)))\
               .map(lambda x: Row(label=0.0, features=Vectors.dense(x[1])))).toDF()"""

        #y_label_noPU = [i.label for i in test_data_4thday.select('label').collect()]

        print(time.asctime())
        log_file.write(time.asctime() + '\n')
        
        ### DecisionTree
        log_file.write("#### DecisionTree ####\n")
        DT_noPU = DecisionTreeClassifier().fit(train_data_noPU)
        DT_predict_4th_day_noPU = DT_noPU.transform(test_data_4thday.select('features'))

        y_predict_noPU_DT = [i.prediction for i in DT_predict_4th_day_noPU.select('prediction').collect()]

        recall_score_noPU_DT = recall_score(y_label_noPU, y_predict_noPU_DT)
        precision_score_noPU_DT = precision_score(y_label_noPU, y_predict_noPU_DT)
        f1_score_noPU_DT = f1_score(y_label_noPU, y_predict_noPU_DT)
        tp_DT = 0
        tn_DT = 0
        for i in range(test_data_4thday_length):
            if y_label_noPU[i] == 1.0 and y_predict_noPU_DT[i] == 1.0:
                tp_DT += 1
            if y_label_noPU[i] == 0.0 and y_predict_noPU_DT[i] == 0.0:
                tn_DT += 1
        accuracy_DT = (tp_DT + tn_DT)/test_data_4thday_length
        
        print(recall_score_noPU_DT, precision_score_noPU_DT, f1_score_noPU_DT, accuracy_DT)

        log_file.write('\n')
        log_file.write("recall_score_noPU_DT: " + str(recall_score_noPU_DT) + '\n')
        log_file.write("precision_score_noPU_DT: " + str(precision_score_noPU_DT) + '\n')
        log_file.write("f1_score_noPU_DT: " + str(f1_score_noPU_DT) + '\n')
        log_file.write("accuracy_DT: " + str(accuracy_DT) + '\n')
        log_file.write(time.asctime() + '\n')
        log_file.write('\n')

        print(time.asctime())
        
        ### NB
        log_file.write("#### NaiveBayes ####\n")
        NB_noPU = NaiveBayes().fit(train_data_noPU)
        NB_predict_4th_day_noPU = NB_noPU.transform(test_data_4thday.select('features'))

        y_predict_noPU_NB = [i.prediction for i in NB_predict_4th_day_noPU.select('prediction').collect()]

        recall_score_noPU_NB = recall_score(y_label_noPU, y_predict_noPU_NB)
        precision_score_noPU_NB = precision_score(y_label_noPU, y_predict_noPU_NB)
        f1_score_noPU_NB = f1_score(y_label_noPU, y_predict_noPU_NB)
        tp_NB = 0
        tn_NB = 0
        for i in range(test_data_4thday_length):
            if y_label_noPU[i] == 1.0 and y_predict_noPU_NB[i] == 1.0:
                tp_NB += 1
            if y_label_noPU[i] == 0.0 and y_predict_noPU_NB[i] == 0.0:
                tn_NB += 1
        accuracy_NB = (tp_NB + tn_NB)/test_data_4thday_length
        
        print(recall_score_noPU_NB, precision_score_noPU_NB, f1_score_noPU_NB, accuracy_NB)

        log_file.write('\n')
        log_file.write("recall_score_noPU_NB: " + str(recall_score_noPU_NB) + '\n')
        log_file.write("precision_score_noPU_NB: " + str(precision_score_noPU_NB) + '\n')
        log_file.write("f1_score_noPU_NB: " + str(f1_score_noPU_NB) + '\n')
        log_file.write("accuracy_NB: " + str(accuracy_NB) + '\n')
        log_file.write(time.asctime() + '\n')
        log_file.write('\n')

        print(time.asctime())
        
        ### LR
        log_file.write("#### LogisticRegression ####\n")
        print("#### LogisticRegression ####\n")
        LR_noPU = LogisticRegression().fit(train_data_noPU)
        
        LR_predict_4th_day_noPU = LR_noPU.transform(test_data_4thday.select('features'))

        y_predict_noPU_LR = [i.prediction for i in LR_predict_4th_day_noPU.select('prediction').collect()]

        recall_score_noPU_LR = recall_score(y_label_noPU, y_predict_noPU_LR)
        precision_score_noPU_LR = precision_score(y_label_noPU, y_predict_noPU_LR)
        f1_score_noPU_LR = f1_score(y_label_noPU, y_predict_noPU_LR)
        tp_LR = 0
        tn_LR = 0
        for i in range(test_data_4thday_length):
            if y_label_noPU[i] == 1.0 and y_predict_noPU_LR[i] == 1.0:
                tp_LR += 1
            if y_label_noPU[i] == 0.0 and y_predict_noPU_LR[i] == 0.0:
                tn_LR += 1
        accuracy_LR = (tp_LR + tn_LR)/test_data_4thday_length
        
        print(recall_score_noPU_LR, precision_score_noPU_LR, f1_score_noPU_LR, accuracy_LR)

        log_file.write('\n')
        log_file.write("recall_score_noPU_LR: " + str(recall_score_noPU_LR) + '\n')
        log_file.write("precision_score_noPU_LR: " + str(precision_score_noPU_LR) + '\n')
        log_file.write("f1_score_noPU_LR: " + str(f1_score_noPU_LR) + '\n')
        log_file.write("accuracy_LR: " + str(accuracy_LR) + '\n')
        log_file.write(time.asctime() + '\n')
        log_file.write('\n')

        print(time.asctime())
        
        ## RandomForestClassifier
        log_file.write("#### RandomForestClassifier ####\n")
        RF_noPU = RandomForestClassifier().fit(train_data_noPU)
        
        RF_predict_4th_day_noPU = RF_noPU.transform(test_data_4thday.select('features'))

        y_predict_noPU_RF = [i.prediction for i in RF_predict_4th_day_noPU.select('prediction').collect()]

        recall_score_noPU_RF = recall_score(y_label_noPU, y_predict_noPU_RF)
        precision_score_noPU_RF = precision_score(y_label_noPU, y_predict_noPU_RF)
        f1_score_noPU_RF = f1_score(y_label_noPU, y_predict_noPU_RF)
        tp_RF = 0
        tn_RF = 0
        for i in range(test_data_4thday_length):
            if y_label_noPU[i] == 1.0 and y_predict_noPU_RF[i] == 1.0:
                tp_RF += 1
            if y_label_noPU[i] == 0.0 and y_predict_noPU_RF[i] == 0.0:
                tn_RF += 1
        accuracy_RF = (tp_RF + tn_RF)/test_data_4thday_length
        
        print(recall_score_noPU_RF, precision_score_noPU_RF, f1_score_noPU_RF, accuracy_RF)

        log_file.write('\n')
        log_file.write("recall_score_noPU_RF: " + str(recall_score_noPU_RF) + '\n')
        log_file.write("precision_score_noPU_RF: " + str(precision_score_noPU_RF) + '\n')
        log_file.write("f1_score_noPU_RF: " + str(f1_score_noPU_RF) + '\n')
        log_file.write("accuracy_RF: " + str(accuracy_RF) + '\n')
        log_file.write(time.asctime() + '\n')
        log_file.write('\n')

        print(time.asctime())

Sat Feb 25 21:22:35 2017
alfa: 0.05
Sat Feb 25 21:22:50 2017
Sat Feb 25 21:22:51 2017
0.99475873437 0.528589806755 0.690346710517 0.5604460428916671
Sat Feb 25 21:26:10 2017
0.773737009517 0.435867302154 0.557614849188 0.39529219234648244
Sat Feb 25 21:27:53 2017
#### LogisticRegression ####

0.974501950987 0.565534198254 0.715715900332 0.61868787780759
Sat Feb 25 21:29:11 2017
0.95544280324 0.827179075523 0.886696504867 0.8797295325810482
Sat Feb 25 21:32:29 2017
Sat Feb 25 21:32:29 2017
alfa: 0.1
Sat Feb 25 21:32:55 2017
Sat Feb 25 21:32:55 2017
0.995222335261 0.509124207808 0.673637049079 0.5250137961218626
Sat Feb 25 21:37:36 2017
0.772925707957 0.435656787811 0.5572318519 0.3949877262088257
Sat Feb 25 21:40:03 2017
#### LogisticRegression ####

0.971295378157 0.57413412499 0.721681346072 0.6309933842045505
Sat Feb 25 21:41:45 2017
0.928219128688 0.843522527794 0.883846403806 0.8798310212936005
Sat Feb 25 21:46:29 2017
Sat Feb 25 21:46:29 2017
alfa: 0.15
Sat Feb 25 21:47:01 2017
Sa

In [15]:
rdd1 = sc.parallelize([1, 2, 3])
rdd2 = sc.parallelize([1, 2, 3])
rdd3 = sc.parallelize([7, 8, 9])

rdd = sc.union([rdd1]*int(0.2/0.05))


In [16]:
rdd.collect()

[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]